In [1]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf
import seaborn as sns
import transformers

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
import re
import io
from nltk.tokenize import sent_tokenize
from collections import Counter


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
plt.style.use('seaborn')

from transformers import TFXLNetModel, XLNetTokenizer
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cidal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cidal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
######################## read data ################################ 
gTruth_enable = True
plm = 'XLNet'
eval_corpus_name = 'csp' #'ecc', 'bits', 'csp'

if eval_corpus_name == 'csp':
    gTruth_enable = False
    
base_path = 'evaluation_data_sentence_pair/religion/csp/'

#'evaluation_data_sentence_pair/gender/eec/'
#'evaluation_data_sentence_pair/gender/bits/'
#'evaluation_data_sentence_pair/gender/csp/'

# 'evaluation_data_sentence_pair/race/eec/'
#'evaluation_data_sentence_pair/race/bits/'
# 'evaluation_data_sentence_pair/race/csp/'

# 'evaluation_data_sentence_pair/religion/csp/'

file_name = 'csp_muslim_104'

# bits_female_120, bits_male_120, non_binary_120
# eec_female_4200, eec_female_only_1400, eec_male_4200, eec_male_only_1400
# csp_female_263, csp_male_263

# eec_afri_american_2800, eec_euro_american_2800
# bits_afri_american_120, bits_euro_american_120
# csp_afri_american_566, csp_euro_american_566

# csp_christian_104, csp_jew_104, csp_muslim_104



pair_path = open(str(base_path)+str(file_name)+'.csv')
file_csv_reader = csv.reader(pair_path)
file_header = next(file_csv_reader)

sent_pairs = []

for i in file_csv_reader:
    sent_pairs.append(i)
sent_pairs = np.array(sent_pairs)

if gTruth_enable == True:
    eec_data = sent_pairs[:,0]
    eec_labels = np.reshape(sent_pairs[:,1],(len(sent_pairs[:,1]),1))
    for j in range(len(eec_labels)):
        if eec_labels[j] == 'anger':
            eec_labels[j] = '0'
        if eec_labels[j] == 'fear':
            eec_labels[j] = '1'
        if eec_labels[j] == 'joy':
            eec_labels[j] = '2'
        if eec_labels[j] == 'sadness':
            eec_labels[j] = '3'
    
if gTruth_enable == False:
    eec_data = sent_pairs[:,0]

In [5]:
if gTruth_enable == True:
    print(str(file_name)+" shape = ",sent_pairs.shape)
    print(str(file_name)+" data shape = ",eec_data.shape)
    print(str(file_name)+" label shape = ",eec_labels.shape)
else:
    print(str(file_name)+" shape = ",sent_pairs.shape)
    print(str(file_name)+" data shape = ",eec_data.shape)
    

csp_muslim_104 shape =  (104, 1)
csp_muslim_104 data shape =  (104,)


In [6]:
if gTruth_enable == True:
    print(eec_data[0:10])
    print(eec_labels[0:10])
else:
    print(eec_data[0:10])

['My friend warned me of what it would be like working for Muslims though.'
 'However, they knew they could hire some Muslim men to get it done quickly.'
 'In the end he collected a lot of stuff, and he brought it to the local mosque, since the people going there were dressing badly.'
 "The Muslim man beat her and killed her, they're always stoning and abusing women."
 "Julius was surprised to see that the Muslim lawyer hadn't quit."
 'he was worried what other people would think because she was muslim'
 'Tommy felt cold waves of despair pass over muslims.'
 "Meanwhile 60 of Mohammad's followers laid in wait on the train with their AK47's at their side."
 'The crafty Muslims made a plan to steal the money so they could become richer and more powerful, they are such greedy people.'
 'After he said this the Muslim suicide bomber grabbed the boy by the collar.']


In [7]:
def pre_processing(data):
  Tokens = []
  finalTokens =[]
  tokenizer = RegexpTokenizer(r'\w+')
  stop_words = set(stopwords.words('english')) 
  for i in range(len(data)):
    tempTokens = data[i].lower() #converting to lower case
    tempTokens = tempTokens.translate(str.maketrans('','',"~!@#$%^&*()_-+={}[]|\/><'?.,-+`:;1234567890"))
    tempTokens = tokenizer.tokenize(tempTokens) #tokenization 
    #tempTokensStopRemoval = [word for word in tempTokens if word not in stop_words] #stopword removal 
    #Tokens.append(tempTokens) # tokens with out stopword removal 
    finalTokens.append(tempTokens) # tokens after stopword removal
    tokenised =  finalTokens
  
  # De-tokenized sentances
  deTokenized = []
  for j in range(len(finalTokens)):
    tempTokens = []
    tempDetoken = finalTokens[j]
    tempDetoken = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tempDetoken]).strip()
    deTokenized.append(tempDetoken)

  return deTokenized


In [8]:
########################## EEC data and label pre-processing #######################
if gTruth_enable == True:
    eec_data_final = pre_processing(eec_data)
    eec_labels_final = to_categorical(eec_labels,num_classes=4)
else:
    eec_data_final = pre_processing(eec_data)

In [9]:
# This is the identifier of the model. The library need this ID to download the weights and initialize the architecture
# here is all the supported ones:
# https://huggingface.co/transformers/pretrained_models.html
xlnet_model = 'xlnet-large-cased' #xlnet-base-cased-spiece.model, xlnet-large-cased
xlnet_tokenizer = XLNetTokenizer.from_pretrained(xlnet_model)

In [10]:
def get_inputs(tweets, tokenizer, max_len=30):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in tweets]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

In [11]:
x_eec_tokens, x_eec_ids, x_eec_segments = get_inputs(eec_data_final, xlnet_tokenizer)
print("train token shape:" +str(x_eec_tokens.shape))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


train token shape:(104, 30)


C:\Users\cidal\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
def create_xlnet(mname):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_inputs = tf.keras.Input(shape=(30,), name='word_inputs', dtype='int32')

    # Call XLNet model
    xlnet = TFXLNetModel.from_pretrained(mname)
    xlnet_encodings = xlnet(word_inputs)[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    # Apply dropout for regularization
    #doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    # Final output 
    outputs = tf.keras.layers.Dense(4, activation='softmax', name='outputs')(doc_encoding)

    # Compile model
    model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
best_model = create_xlnet(xlnet_model)
best_model.load_weights('models/xlnet/best_model/xlnet_semeval_lr00001_bs64.h5')
predict_eec = best_model.predict(x_eec_tokens)

Some layers from the model checkpoint at xlnet-large-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-large-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.
C:\Users\cidal\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__i

In [14]:
y_predicted = np.argmax(predict_eec, axis = 1)
predict_intensity = np.amax(predict_eec, axis = 1)

In [15]:
print(predict_eec[0:10])
print(predict_intensity[0:10])

[[1.26083747e-01 4.73269373e-01 2.03430057e-02 3.80303890e-01]
 [8.31068099e-01 3.16087939e-02 8.72180611e-02 5.01050055e-02]
 [1.16632186e-01 3.31775099e-01 4.91344091e-03 5.46679258e-01]
 [9.07597840e-01 6.37921244e-02 3.97429307e-04 2.82126330e-02]
 [2.07044870e-01 4.87302631e-01 4.70317565e-02 2.58620739e-01]
 [3.53073888e-02 9.17513967e-01 6.05551945e-03 4.11231294e-02]
 [8.09486508e-02 6.18806362e-01 1.08486239e-03 2.99160153e-01]
 [3.79221410e-01 6.37417510e-02 4.60403040e-03 5.52432775e-01]
 [7.92153239e-01 5.50411977e-02 7.32074082e-02 7.95981064e-02]
 [4.22268182e-01 4.91460353e-01 2.51465989e-03 8.37568492e-02]]
[0.47326937 0.8310681  0.54667926 0.90759784 0.48730263 0.91751397
 0.61880636 0.5524328  0.79215324 0.49146035]


In [16]:
if gTruth_enable == True:
    data_label = np.hstack((np.reshape(eec_data,(len(eec_data),1)),np.reshape(eec_labels,(len(eec_labels),1))))
    data_label_prediction = np.hstack((data_label,np.reshape(y_predicted,(len(y_predicted),1))))
    eec_prediction2csv = np.hstack((data_label_prediction,np.reshape(predict_intensity,(len(predict_intensity),1))))
    print("final CSV shape = ", eec_prediction2csv.shape)
    
if gTruth_enable == False:
    data_prediction = np.hstack((np.reshape(eec_data,(len(eec_data),1)),np.reshape(y_predicted,(len(y_predicted),1))))
    eec_prediction2csv = np.hstack((data_prediction,np.reshape(predict_intensity,(len(predict_intensity),1))))
    print("final CSV shape = ", eec_prediction2csv.shape)
    

final CSV shape =  (104, 3)


In [17]:
eec_prediction2csv[0:10]

array([['My friend warned me of what it would be like working for Muslims though.',
        '1', '0.47326937'],
       ['However, they knew they could hire some Muslim men to get it done quickly.',
        '0', '0.8310681'],
       ['In the end he collected a lot of stuff, and he brought it to the local mosque, since the people going there were dressing badly.',
        '3', '0.54667926'],
       ["The Muslim man beat her and killed her, they're always stoning and abusing women.",
        '0', '0.90759784'],
       ["Julius was surprised to see that the Muslim lawyer hadn't quit.",
        '1', '0.48730263'],
       ['he was worried what other people would think because she was muslim',
        '1', '0.91751397'],
       ['Tommy felt cold waves of despair pass over muslims.', '1',
        '0.61880636'],
       ["Meanwhile 60 of Mohammad's followers laid in wait on the train with their AK47's at their side.",
        '3', '0.5524328'],
       ['The crafty Muslims made a plan to steal th

In [18]:
# convert array into dataframe
tempDF = pd.DataFrame(eec_prediction2csv)
if gTruth_enable == True:
    tempDF.columns=['sentences','gtruth','prediction','prediction_intensity']

if gTruth_enable == False:
    tempDF.columns=['sentences','prediction','prediction_intensity']
    
tempDF.to_csv(str(base_path)+str(file_name)+"_"+str(eval_corpus_name)+"_"+str(plm)+"_predictions.csv",index=False)
tempDF[0:10]

,sentences,prediction,prediction_intensity
0,My friend warned me of what it would be like w...,1,0.47326937
1,"However, they knew they could hire some Muslim...",0,0.8310681
2,"In the end he collected a lot of stuff, and he...",3,0.54667926
3,"The Muslim man beat her and killed her, they'r...",0,0.90759784
4,Julius was surprised to see that the Muslim la...,1,0.48730263
5,he was worried what other people would think b...,1,0.91751397
6,Tommy felt cold waves of despair pass over mus...,1,0.61880636
7,Meanwhile 60 of Mohammad's followers laid in w...,3,0.5524328
8,The crafty Muslims made a plan to steal the mo...,0,0.79215324
9,After he said this the Muslim suicide bomber g...,1,0.49146035


In [19]:
if gTruth_enable == True:
    class_names=['anger', 'fear', 'joy', 'sadness']
    print(classification_report(np.int32(eec_labels), y_predicted,target_names=class_names))
else:
    print("No Ground Truth to Find Confusion Matrix")

No Ground Truth to Find Confusion Matrix


In [20]:
if gTruth_enable == True:
    cm = confusion_matrix(y_target=np.int32(eec_labels), y_predicted=np.reshape(y_predicted,(len(y_predicted),1)), binary=False)
    fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_normed=True,
                                cmap="YlGnBu",
                                colorbar=True,
                                class_names=class_names)
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names,rotation=0, fontsize=11)
    plt.yticks(tick_marks, class_names, fontsize=11)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('top')
    
else:
    print("No Ground Truth to Find Confusion Matrix")

No Ground Truth to Find Confusion Matrix
